In [1]:
# 0 := delete \n
# 1 := leave as it is

# model = gensim.models.Word2Vec.load('../../../pretrained_model/kor/ko.bin')

# from hangul_utils import split_syllables, join_jamos
# import gensim

In [2]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data
import bpe
import utils
import pretrained_model as pm
import data_loader as dl
import trainer
import initializer as init
import tester
import model_util as mu

In [3]:
first_np, second_np, label_np  = data.getData(0)

no_first_np, no_second_np, no_label_np  = data.getData(1)

In [4]:
first_np, second_np = utils.process_splitted(first_np, second_np)

no_first_np, no_second_np = utils.process_splitted(no_first_np, no_second_np)

linearized complete!
linearized complete!


In [5]:
first_ls, second_ls, ch2idx, max_len = utils.tokenize(first_np, second_np)
first2idx_np, second2idx_np = utils.encode(first_ls, second_ls, ch2idx, max_len)

no_first_ls, no_second_ls, no_ch2idx, no_max_len = utils.tokenize(no_first_np, no_second_np)
no_first2idx_np, no_second2idx_np = utils.encode(no_first_ls, no_second_ls, no_ch2idx, no_max_len)

done tokenizing both data!
saved ch2idx to file!
encoding comlete!
done tokenizing both data!
saved ch2idx to file!
encoding comlete!


In [6]:
# pretrained_word2vec = pm.load_pretrained_model(ch2idx)
# pretrained_word2vec = torch.tensor(pretrained_word2vec)

There are 27 / 133 pretrained vectors found.


In [7]:
train_first, test_first, train_second, test_second, train_labels, test_labels = train_test_split(
    first2idx_np, second2idx_np, label_np, test_size = 0.1, random_state = 43
)

train_first, val_first, train_second, val_second, train_labels, val_labels = train_test_split(
    train_first, train_second, train_labels, test_size = 0.1, random_state = 43
)

no_train_first, no_test_first, no_train_second, no_test_second, no_train_labels, no_test_labels = train_test_split(
    no_first2idx_np, no_second2idx_np, no_label_np, test_size = 0.1, random_state = 43
)

no_train_first, no_val_first, no_train_second, no_val_second, no_train_labels, no_val_labels = train_test_split(
    no_train_first, no_train_second, no_train_labels, test_size = 0.1, random_state = 43
)

In [8]:
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(
                                                                   train_first,
                                                                   train_second,
                                                                   val_first,
                                                                   val_second,
                                                                   test_first,
                                                                   test_second,
                                                                   train_labels,
                                                                   val_labels,
                                                                   test_labels,
                                                                   batch_size=100)

no_train_dataloader, no_val_dataloader, no_test_dataloader = dl.data_loader(
                                                                   no_train_first,
                                                                   no_train_second,
                                                                   no_val_first,
                                                                   no_val_second,
                                                                   no_test_first,
                                                                   no_test_second,
                                                                   no_train_labels,
                                                                   no_val_labels,
                                                                   no_test_labels,
                                                                   batch_size=100)

In [9]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/final/split')

In [10]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [11]:
# ====================
# set parameters here
# ====================

title = 'split_test_1'
epochs = 40

vocab_size=len(ch2idx)
embed_dim = 100
hidden_size = 200
num_classes = 2
rnn_layers = 1

num_filters = [100, 200, 100]
kernel_sizes = [15, 21, 114]

dropout = 0.2

learning_rate = 0.001
weight_decay = 1e-4

model_name="RNN"
optim_name="Adam"
loss_fn_name="CEL"

pretrained_model=None
freeze_embedding=False,

In [12]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(model)

doing with pretrained model!!!
OCR_rnn(
  (emb): Embedding(133, 200)
  (lstm1): RNN(
    (rnn): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=400, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.0, inplace=False)
  (dp2): Dropout(p=0.0, inplace=False)
)


In [13]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title+'yes_gudu',
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.236924   | 91.004115  | 0.163169 | 94.70  | 70.05 
   2    |   0.124648   | 95.481481  | 0.116488 | 96.11  | 68.64 
   3    |   0.097195   | 96.506173  | 0.098676 | 96.85  | 68.49 
   4    |   0.076764   | 97.288066  | 0.081948 | 97.00  | 68.74 
   5    |   0.066478   | 97.514403  | 0.086427 | 97.00  | 68.47 
   6    |   0.058285   | 97.946502  | 0.085777 | 96.70  | 70.24 
   7    |   0.051059   | 98.078189  | 0.075075 | 97.33  | 69.58 
   8    |   0.043834   | 98.403292  | 0.077979 | 97.56  | 72.06 
   9    |   0.041250   | 98.518519  | 0.087841 | 97.19  | 72.10 
  10    |   0.032298   | 98.843621  | 0.073635 | 97.74  | 71.92 
  11    |   0.028712   | 98.938272  | 0.090851 | 97.15  | 71.74 
  12    |   0.024686   | 99.061728  | 0.101292 | 97.15  | 72.77 
  13    |   0.017271   | 99.395062  | 0.112292 | 97.1

In [14]:
mu.saveModel(title+'yes_gudu', model)

In [15]:
model = mu.getModel(title+'yes_gudu')
print(model)

OCR_rnn(
  (emb): Embedding(133, 200)
  (lstm1): RNN(
    (rnn): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=400, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.0, inplace=False)
  (dp2): Dropout(p=0.0, inplace=False)
)


In [16]:
loss, acc = tester.test(test_dataloader=no_test_dataloader,
                        device=device,
                        model=model,
                        title=title+'yes_gudu')

test loss:  3.0156676789124806
test acc:  80.2
saved precision and recall results to file!


In [17]:
with open('../result/final_split', 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
        f.write(text)

In [18]:
mu.graphModel(train_dataloader, model, writer)

uploaded model graph to tensorboard!


In [19]:
trainer.set_seed(42)

no_model, no_optimizer, no_loss_fn = init.initialize_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(no_model)

doing with pretrained model!!!
OCR_rnn(
  (emb): Embedding(133, 200)
  (lstm1): RNN(
    (rnn): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=400, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.0, inplace=False)
  (dp2): Dropout(p=0.0, inplace=False)
)


In [20]:

start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title+'no_gudu',
    writer=writer,
    train_dataloader=no_train_dataloader,
    val_dataloader=no_val_dataloader,
    device=device,
    model=no_model,
    optimizer=no_optimizer,
    loss_fn=no_loss_fn
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.312176   | 87.419753  | 0.225479 | 91.56  | 68.37 
   2    |   0.188457   | 93.004115  | 0.179643 | 93.19  | 68.51 
   3    |   0.139370   | 94.798354  | 0.134548 | 94.96  | 68.48 
   4    |   0.115777   | 95.584362  | 0.120019 | 95.37  | 68.42 
   5    |   0.097988   | 96.316872  | 0.137769 | 94.74  | 68.16 
   6    |   0.082520   | 96.934156  | 0.117215 | 96.22  | 68.51 
   7    |   0.068715   | 97.415638  | 0.100947 | 96.30  | 69.12 
   8    |   0.059495   | 97.777778  | 0.107078 | 96.07  | 69.24 
   9    |   0.051698   | 98.123457  | 0.127317 | 95.78  | 68.92 
  10    |   0.046423   | 98.205761  | 0.118974 | 96.19  | 68.44 
  11    |   0.036776   | 98.658436  | 0.110807 | 96.41  | 68.02 
  12    |   0.033592   | 98.860082  | 0.125282 | 95.93  | 68.02 
  13    |   0.028101   | 98.987654  | 0.143943 | 96.0

In [31]:
mu.saveModel(title+'no_gudu', no_model)

In [32]:
no_model = mu.getModel(title+'no_gudu')
print(no_model)

OCR_rnn(
  (emb): Embedding(133, 200)
  (lstm1): RNN(
    (rnn): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=400, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.0, inplace=False)
  (dp2): Dropout(p=0.0, inplace=False)
)


In [34]:
loss, acc = tester.test(test_dataloader=no_test_dataloader,
                        device=device,
                        model=no_model,
                        title=title+'no_gudu')

test loss:  0.20598777152918046
test acc:  97.36666666666667
saved precision and recall results to file!


In [35]:
with open('../result/final_split', 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
        f.write(text)

In [36]:
mu.graphModel(no_train_dataloader, no_model, writer)

uploaded model graph to tensorboard!
